In [1]:
import torch
import numpy as np
import pytorch_lightning as pl
import torchvision.transforms as tf
from tqdm import tqdm
from predict import *
from HIST2ST import *
from dataset import ViT_HER2ST, ViT_SKIN
from scipy.stats import pearsonr,spearmanr
from torch.utils.data import DataLoader
from pytorch_lightning.loggers import TensorBoardLogger
from copy import deepcopy as dcp
from collections import defaultdict as dfd
from sklearn.metrics import adjusted_rand_score as ari_score
from sklearn.metrics.cluster import normalized_mutual_info_score as nmi_score

/opt/anaconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data Loading

In [2]:
name=[*[f'A{i}' for i in range(2,7)],*[f'B{i}' for i in range(1,7)],
      *[f'C{i}' for i in range(1,7)],*[f'D{i}' for i in range(1,7)],
      *[f'E{i}' for i in range(1,4)],*[f'F{i}' for i in range(1,4)],*[f'G{i}' for i in range(1,4)]]
patients = ['P2', 'P5', 'P9', 'P10']
reps = ['rep1', 'rep2', 'rep3']
skinname = []
for i in patients:
    for j in reps:
        skinname.append(i+'_ST_'+j)
device='cuda'
tag='5-7-2-8-4-16-32'
k,p,d1,d2,d3,h,c=map(lambda x:int(x),tag.split('-'))
dropout=0.2
random.seed(12000)
np.random.seed(12000)
torch.manual_seed(12000)
torch.cuda.manual_seed(12000)
torch.cuda.manual_seed_all(12000)  
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True


# Hist2ST Prediction

### To run the trained model, please select the trained model and replace the value of the variable fold with the number in the name of the selected trained model.

In [3]:
fold=0
data='her2st'
prune='Grid' if data=='her2st' else 'NA'
genes=785
# genes=171 if data=='cscc' else 785

In [9]:
testset = pk_load(fold,'test',dataset=data,flatten=False,adj=True,ori=True,prune=prune)
test_loader = DataLoader(testset, batch_size=1, num_workers=0, shuffle=False)
label=testset.label[testset.names[0]]
genes=785
model=Hist2ST(
    depth1=d1, depth2=d2,depth3=d3,n_genes=genes, 
    kernel_size=k, patch_size=p,
    heads=h, channel=c, dropout=0.2,
    zinb=0.25, nb=False,
    bake=5, lamb=0.5, 
)
model.load_state_dict(torch.load(f'./model/{fold}-Hist2ST.ckpt'))
pred, gt = test(model, test_loader,'cuda')

['ME']
Loading imgs...
Loading metadata...


KeyError: "['PRXL2B', 'SLC66A1', 'MICOS10', 'P3R3URF', 'TMEM275', 'CZIB', 'CCN1', 'DIPK1A', 'TLCD4', 'TAFA3', 'AL162596.1', 'EEF1AKNMT', 'NIBAN1', 'RO60', 'PACC1', 'SLC66A3', 'LRATD1', 'FAM166C', 'PCARE', 'ECRG4', 'SEPTIN10', 'MTLN', 'ITPRID2', 'FLACC1', 'SPEGNB', 'MAB21L4', 'SEPTIN2', 'FAM240C', 'GASK1A', 'PRSS45P', 'TASOR', 'TAFA1', 'TAFA4', 'TEX55', 'SLC49A4', 'PRR20G', 'DIPK2A', 'STRIT1', 'SLC66A1L', 'AC072022.2', 'PLAAT1', 'NCBP2AS2', 'TMEM271', 'SEPTIN11', 'GASK1B', 'ATPSCKMT', 'AC106774.4', 'TAF11L4', 'TAF11L6', 'TAF11L8', 'TAF11L11', 'TAF11L12', 'TAF11L13', 'TAF11L14', 'CERT1', 'MINAR2', 'SEPTIN8', 'INSYN2B', 'GMCL2', 'AC113348.1', 'SNHG32', 'TSBP1', 'ILRUN', 'BNIP5', 'MMUT', 'AL135905.2', 'DOP1A', 'MTRES1', 'CCN6', 'CCN2', 'RAMACL', 'CRPPA', 'NUP42', 'ITPRID1', 'SEPTIN7', 'YAE1', 'CCDC201', 'CDC14C', 'SEPTIN14', 'DENND11', 'AC073111.4', 'AC134684.8', 'RBIS', 'CCN3', 'LRATD2', 'CCN4', 'DOCK8-AS1', 'PRXL2C', 'SHOC1', 'NIBAN2', 'DIPK1B', 'AKR1C8P', 'TASOR2', 'MIR1915HG', 'AC06775

In [ ]:
R=get_R(pred,gt)[0]
print('Pearson Correlation:',np.nanmean(R))


# clus,ARI=cluster(pred,label)
# print('ARI:',ARI)


Pearson Correlation: 0.2887870599966082
ARI: 0.431
